
# Natural Language Processing 📝 — A Complete Guide (CodSoft Task 4: Spam SMS Detection)

**Author:** Your Name  
**Date:** 2025-08-26  
**Batch:** AUGUST B43

This hands-on notebook walks through an end‑to‑end NLP workflow using the classic **SMS Spam Collection** dataset:
- Loading data and quick EDA
- Text cleaning, tokenization, (optional) stemming
- Bag‑of‑Words and **TF‑IDF**
- Models: **Naive Bayes**, **Logistic Regression**, **Linear SVM**
- Evaluation: Accuracy, Precision, Recall, F1, Confusion Matrix, ROC‑AUC
- Save a reusable pipeline and run inference on new messages
- *(Optional bonus)* LSTM and BERT stubs for you to extend locally

> ⚠️ **Internship requirements** (remember for submission): maintain a `CODSOFT` GitHub repo, keep code original, and share a short LinkedIn demo tagging `@CodSoft` with `#codsoft`.


In [ ]:

# === Setup ===
# This notebook uses only standard libraries available in most Python setups.
# If something is missing, uncomment and install locally.
# !pip install -U scikit-learn pandas numpy matplotlib joblib

import os, re, json, math, random, string, joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (classification_report, confusion_matrix, roc_curve,
                             auc, precision_recall_fscore_support, accuracy_score)

# Set a seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

print("Versions → pandas:", pd.__version__, "| numpy:", np.__version__)


## 1. Loading Data 💎

In [ ]:

# Path to the dataset. For CodSoft's spam task, typical file name is 'spam.csv'.
# If running on Kaggle/Colab, adjust the path accordingly.
CSV_PATH = 'spam.csv'  # change if needed

# Try reading with common encodings
df = None
for enc in ['latin-1', 'utf-8', None]:
    try:
        df = pd.read_csv(CSV_PATH, encoding=enc) if enc else pd.read_csv(CSV_PATH)
        break
    except Exception as e:
        print(f"Read failed with encoding={enc}: {e}")

if df is None:
    raise FileNotFoundError("Could not read spam.csv. Make sure the file is in the working directory.")

# The canonical dataset often has these columns: v1 (label), v2 (message), plus unnamed extras.
if set(['v1','v2']).issubset(df.columns):
    df = df[['v1','v2']].rename(columns={'v1':'label','v2':'text'})
else:
    # fallback: use first two columns as (label, text)
    df = df.iloc[:, :2]
    df.columns = ['label','text']

# Basic cleanup
df = df.dropna(subset=['label','text']).drop_duplicates(subset=['label','text'])
df['label'] = df['label'].str.strip().str.lower()
df = df[df['label'].isin(['ham','spam'])].copy()

# Add message length feature (number of tokens)
df['message_len'] = df['text'].apply(lambda s: len(str(s).split()))
df.head()


## 2. EDA 📊

In [ ]:

# Class distribution
counts = df['label'].value_counts().sort_index()
print(counts)

fig = plt.figure()
plt.bar(counts.index, counts.values)
plt.title('Class Distribution (ham vs spam)')
plt.xlabel('Class')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

# Message length distribution (separate charts to keep one-plot-per-figure)
fig = plt.figure()
plt.hist(df[df['label']=='ham']['message_len'], bins=40)
plt.title('Message Lengths – HAM')
plt.xlabel('Tokens per message')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

fig = plt.figure()
plt.hist(df[df['label']=='spam']['message_len'], bins=40)
plt.title('Message Lengths – SPAM')
plt.xlabel('Tokens per message')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

print("Mean length | ham:", df[df.label=='ham']['message_len'].mean(),
      "| spam:", df[df.label=='spam']['message_len'].mean())


## 3. Data Preprocessing ⚙️

In [ ]:

# We avoid external downloads by using scikit-learn's ENGLISH_STOP_WORDS.
CUSTOM_STOP = set(list(ENGLISH_STOP_WORDS) + ['u','im','c'])

URL_RE   = re.compile(r'(https?://\S+|www\.\S+)')
EMAIL_RE = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w+\b')
NUM_RE   = re.compile(r'\b\d+\b')
NONALNUM = re.compile(r'[^a-z0-9\s]')

def clean_text(text: str) -> str:
    """Lowercase, replace URLs/emails/numbers, strip punctuation and extra spaces."""
    text = str(text).lower()
    text = URL_RE.sub(' URL ', text)
    text = EMAIL_RE.sub(' EMAIL ', text)
    text = NUM_RE.sub(' NUM ', text)
    text = NONALNUM.sub(' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Optional stemming (no NLTK download needed if package is present; otherwise skipped)
try:
    import nltk
    from nltk.stem import SnowballStemmer
    STEMMER = SnowballStemmer('english')
except Exception:
    STEMMER = None
    print('NLTK not available → skipping stemming.')

def remove_stopwords(text: str) -> str:
    return ' '.join(tok for tok in text.split() if tok not in CUSTOM_STOP)

def stem_text(text: str) -> str:
    if STEMMER is None:
        return text
    return ' '.join(STEMMER.stem(tok) for tok in text.split())

def preprocess_pipeline(text: str) -> str:
    text = clean_text(text)
    text = remove_stopwords(text)
    text = stem_text(text)  # harmless if STEMMER is None
    return text

# Create a preview column
df['text_clean'] = df['text'].apply(preprocess_pipeline)
df[['label','text','text_clean']].head(8)


## 4. Tokens visualization 📊

In [ ]:

# CountVectorizer on cleaned text for quick top-token stats
cv = CountVectorizer(min_df=2)
X_counts = cv.fit_transform(df['text_clean'])
vocab = np.array(cv.get_feature_names_out())

# Top tokens overall
total_counts = np.asarray(X_counts.sum(axis=0)).ravel()
top_idx = np.argsort(total_counts)[-20:][::-1]
top_tokens = list(zip(vocab[top_idx], total_counts[top_idx]))
print("Top 20 tokens overall:") 
for t,c in top_tokens: 
    print(f"{t:>15s} : {c}")

# Plot frequencies
fig = plt.figure(figsize=(10,5))
plt.bar(range(len(top_tokens)), [c for _,c in top_tokens])
plt.xticks(range(len(top_tokens)), [t for t,_ in top_tokens], rotation=45, ha='right')
plt.title('Top 20 Tokens Overall')
plt.tight_layout()
plt.show()


## 5. Train/Test split

In [ ]:

X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)
len(X_train), len(X_test)


## 6. Modeling

In [ ]:

# Build three comparable pipelines using TF‑IDF + classic ML
pipelines = {
    'NaiveBayes': Pipeline([
        ('tfidf', TfidfVectorizer(preprocessor=preprocess_pipeline, stop_words='english', ngram_range=(1,2), min_df=2)),
        ('clf', MultinomialNB(alpha=0.5))
    ]),
    'LogisticRegression': Pipeline([
        ('tfidf', TfidfVectorizer(preprocessor=preprocess_pipeline, stop_words='english', ngram_range=(1,2), min_df=2)),
        ('clf', LogisticRegression(max_iter=2000, solver='liblinear'))
    ]),
    'LinearSVM': Pipeline([
        ('tfidf', TfidfVectorizer(preprocessor=preprocess_pipeline, stop_words='english', ngram_range=(1,2), min_df=2)),
        ('clf', CalibratedClassifierCV(LinearSVC(), cv=5))
    ]),
}

metrics_rows = []
reports = {}
cms = {}

for name, pipe in pipelines.items():
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    p, r, f1, _ = precision_recall_fscore_support(y_test, y_pred, pos_label='spam', average='binary', zero_division=0)
    metrics_rows.append({'model': name, 'accuracy': round(acc,4), 'precision_spam': round(p,4),
                         'recall_spam': round(r,4), 'f1_spam': round(f1,4)})
    reports[name] = classification_report(y_test, y_pred, digits=4, zero_division=0)
    cms[name] = confusion_matrix(y_test, y_pred, labels=['ham','spam'])

results = pd.DataFrame(metrics_rows).sort_values('f1_spam', ascending=False).reset_index(drop=True)
results


In [ ]:

best_model_name = results.iloc[0]['model']
best_pipe = pipelines[best_model_name]

print("Best model:", best_model_name)
print("\nClassification report:\n", reports[best_model_name])

# Confusion matrix
cm = cms[best_model_name]
fig = plt.figure()
plt.imshow(cm, interpolation='nearest')
plt.title(f'Confusion Matrix – {best_model_name}')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks([0,1], ['ham','spam'])
plt.yticks([0,1], ['ham','spam'])
for (i, j), z in np.ndenumerate(cm):
    plt.text(j, i, f"{z}", ha='center', va='center')
plt.tight_layout()
plt.show()

# ROC curve using Logistic Regression (for smooth probability estimates)
lr_pipe = pipelines['LogisticRegression']
y_proba_lr = lr_pipe.predict_proba(X_test)[:, list(lr_pipe.classes_).index('spam')]
fpr, tpr, thr = roc_curve((y_test=='spam').astype(int), y_proba_lr)
roc_auc = auc(fpr, tpr)

fig = plt.figure()
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
plt.plot([0,1], [0,1], linestyle='--')
plt.title('ROC Curve – Logistic Regression (spam vs. ham)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()


## 7. Save the best pipeline & Predict on new text

In [ ]:

# Save
MODEL_PATH = f"spam_sms_{best_model_name.lower()}_tfidf_pipeline.joblib"
joblib.dump(best_pipe, MODEL_PATH)
print("Saved:", MODEL_PATH)

# Quick demo
demo_texts = [
    "Congratulations! You have won a $1000 gift card. Click http://bit.ly/xyz to claim now!", 
    "Are we still on for dinner tonight at 7?", 
    "URGENT: Your account was locked due to suspicious activity. Verify at www.example.com", 
    "hey, call me when you're free"
]
preds = best_pipe.predict(demo_texts)
for t, p in zip(demo_texts, preds):
    print(f"[{p.upper()}] {t}")


## 8. (Optional) LSTM — extend locally

In [ ]:

RUN_DEEP = False  # set True locally if you want to train a quick LSTM

if RUN_DEEP:
    try:
        import tensorflow as tf
        from tensorflow.keras.preprocessing.text import Tokenizer
        from tensorflow.keras.preprocessing.sequence import pad_sequences
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPool1D

        # Tokenize cleaned text
        texts = df['text_clean'].tolist()
        y_bin = (df['label']=='spam').astype(int).values

        tok = Tokenizer(num_words=10000, oov_token='[UNK]')
        tok.fit_on_texts(texts)
        seqs = tok.texts_to_sequences(texts)
        maxlen = max(len(s) for s in seqs)
        X_pad = pad_sequences(seqs, maxlen=min(maxlen, 120), padding='post', truncating='post')

        X_tr, X_te, y_tr, y_te = train_test_split(X_pad, y_bin, test_size=0.2, random_state=RANDOM_STATE, stratify=y_bin)

        model = Sequential([
            Embedding(input_dim=min(10000, len(tok.word_index)+1), output_dim=64, input_length=X_tr.shape[1]),
            LSTM(64, return_sequences=True),
            GlobalMaxPool1D(),
            Dropout(0.3),
            Dense(64, activation='relu'),
            Dropout(0.3),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        model.summary()
        history = model.fit(X_tr, y_tr, validation_data=(X_te, y_te), epochs=2, batch_size=32)
    except Exception as e:
        print("Skipping LSTM due to error:", e)
else:
    print("LSTM section is fully coded but skipped (RUN_DEEP=False). Enable it locally to train.")


## 9. (Optional) BERT — extend locally

In [ ]:

# This is a light stub you can enable locally if you have 'transformers' installed.
RUN_BERT = False  # set True locally to try a tiny BERT pipeline

if RUN_BERT:
    try:
        from transformers import BertTokenizerFast, TFBertForSequenceClassification
        import tensorflow as tf

        texts = df['text_clean'].tolist()
        y_bin = (df['label']=='spam').astype(int).values
        X_tr, X_te, y_tr, y_te = train_test_split(texts, y_bin, test_size=0.2, random_state=RANDOM_STATE, stratify=y_bin)

        tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
        tr_enc = tokenizer(X_tr, truncation=True, padding=True, max_length=128, return_tensors='tf')
        te_enc = tokenizer(X_te, truncation=True, padding=True, max_length=128, return_tensors='tf')

        model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])
        history = model.fit(tr_enc.data, y_tr, validation_data=(te_enc.data, y_te), epochs=1, batch_size=8)
    except Exception as e:
        print("Skipping BERT due to error:", e)
else:
    print("BERT section is fully coded but skipped (RUN_BERT=False). Enable it locally to train.")



## 10. Wrap‑up

- You now have an end‑to‑end, **original** NLP pipeline for SMS spam detection.
- For submission:
  1. Push this notebook and the saved `.joblib` model to your **`CODSOFT`** GitHub repo.
  2. Record a short demo video and post on LinkedIn tagging **@CodSoft** with **#codsoft**.
  3. Keep your code clean, documented, and **do not copy** code verbatim from others.

> Tip: For production, wrap the saved pipeline in a small REST/Streamlit app to allow interactive predictions.


## 11. (Optional) Streamlit Demo 🎛

In [ ]:

# A minimal Streamlit demo app for Spam SMS Detection
# Save this snippet as app.py and run with: streamlit run app.py

import streamlit as st
import joblib

st.title("📱 Spam SMS Detector")
st.write("Type an SMS below to check if it's Spam or Ham.")

# Load pipeline
pipe = joblib.load("spam_sms_naivebayes_tfidf_pipeline.joblib")

user_input = st.text_area("Enter SMS text:", "")

if st.button("Predict"):
    if user_input.strip():
        pred = pipe.predict([user_input])[0]
        st.success(f"Prediction: **{pred.upper()}**")
    else:
        st.warning("Please enter some text.")



## 11. Interactive App (Streamlit) 🚀

Run a small web app for live predictions.

**How to run locally (terminal):**
```bash
pip install streamlit joblib scikit-learn
streamlit run streamlit_spam_app.py
```


In [ ]:

# This cell writes a Streamlit app to the current directory.
# After running this cell locally, launch it with:
#    streamlit run streamlit_spam_app.py

import os, joblib, textwrap

APP_PATH = "streamlit_spam_app.py"
MODEL_GLOB = None

# Try to infer the saved model name from this notebook run (fallback to any .joblib file).
for f in os.listdir():
    if f.endswith(".joblib") and "spam_sms_" in f:
        MODEL_GLOB = f
        break
if MODEL_GLOB is None:
    # fallback: a generic name (user can change in app UI)
    MODEL_GLOB = "spam_sms_naivebayes_tfidf_pipeline.joblib"

app_code = f"""
import os
import streamlit as st
import joblib
import numpy as np

st.set_page_config(page_title="Spam SMS Detector", page_icon="📱", layout="centered")

st.title("📱 Spam SMS Detector")
st.caption("TF-IDF + Classic ML (CodSoft Task 4)")

@st.cache_resource
def load_pipeline(path: str):
    return joblib.load(path)

# Sidebar: model selection
st.sidebar.header("Model")
default_model_path = "{MODEL_GLOB}"
model_file = st.sidebar.text_input("Path to .joblib pipeline", value=default_model_path)
uploaded = st.sidebar.file_uploader("...or upload a .joblib file", type=["joblib"])

pipeline = None
if uploaded is not None:
    tmp_path = "uploaded_pipeline.joblib"
    with open(tmp_path, "wb") as f:
        f.write(uploaded.read())
    model_file = tmp_path

try:
    pipeline = load_pipeline(model_file)
    st.sidebar.success(f"Loaded: {model_file}")
except Exception as e:
    st.sidebar.warning("Could not load pipeline. Use the text box or upload a .joblib file.")

st.subheader("Try it out")
sms = st.text_area("Paste an SMS message here:", height=120, placeholder="e.g., Congratulations! You won a prize. Claim at http://bit.ly/...")

def predict_one(text: str):
    if pipeline is None:
        return None, None
    pred = pipeline.predict([text])[0]
    prob = None
    # Try to show probabilities if available
    if hasattr(pipeline, "predict_proba"):
        try:
            proba = pipeline.predict_proba([text])[0]
            # proba is [P(ham), P(spam)] or similar; find 'spam' index
            classes = list(getattr(pipeline, "classes_", []))
            if classes and "spam" in classes:
                prob = float(proba[classes.index("spam")])
            else:
                # fallback to max prob
                prob = float(np.max(proba))
        except Exception:
            prob = None
    return pred, prob

col1, col2 = st.columns([1,1])
with col1:
    if st.button("Predict"):
        if not sms.strip():
            st.info("Please paste a message first.")
        else:
            label, prob = predict_one(sms)
            if label is None:
                st.error("Pipeline not loaded.")
            else:
                st.markdown(f"**Prediction:** :{'green_circle' if label=='ham' else 'red_circle'}: **{label.upper()}**")
                if prob is not None:
                    st.write(f"Spam probability: **{prob:.3f}**")
with col2:
    st.write("Examples")
    samples = [
        "Congratulations! You have won a $1000 gift card. Click http://bit.ly/xyz to claim now!",
        "Are we still on for dinner tonight at 7?",
        "URGENT: Your account was locked due to suspicious activity. Verify at www.security-check.com",
        "hey, call me when you're free",
    ]
    if st.button("Run examples"):
        for s in samples:
            label, prob = predict_one(s)
            st.write(f"- **{label.upper() if label else 'N/A'}** — {s}")
            if prob is not None:
                st.caption(f"Spam probability: {prob:.3f}")

st.sidebar.markdown("—")
st.sidebar.markdown("**Tips**")
st.sidebar.markdown("- Use the same preprocessing vectorizer embedded in your saved pipeline.")
st.sidebar.markdown("- For consistent results, use the `.joblib` produced by this notebook.")
"""

with open(APP_PATH, "w", encoding="utf-8") as f:
    f.write(app_code)

print("Wrote:", APP_PATH)
